In this problem, you will implement a simplified version of the "attention mechanism".

# A simplified version of the "attention mechanism"
- The first section will describe the data.
- The second section will describe the model.
- The third section will be the `ag.Scalar` library
- The fourth section will describe the problem itself.

## The data

We consider a setting where *each* sample $x^{(i)}$, where $i \in \{1,\dots, n\}$ (below, we take $n=5$, is itself a *collection* of (row) $C$ vectors (below, we will take $C = 4$), i.e.,
$$
x^{(i)}
=
\{x^{(i,1)}, \dots, x^{(i,C)}  \}  \quad \text{where} \quad x^{(i,c)} \in \mathbb{R}^d \quad \text{for each} \quad c =1,\dots, C.
$$

For a fixed $i$, we think of the $x^{(i,j)} \in \mathbb{R}^d$ (row) vector as a "word embedding". (Below, we will take $d = 3$.) So the collection $\{x^{(i,1)}, \dots, x^{(i,C)}  \}$ can be viewed as a "sentence embedding" where each sentence has $C$ words.

Alternatively, you can also view each $x^{(i)}$ as a matrix whose $c$-th row is $x^{(i,c)}$, i.e.,

$$
X^{(i)}
=
\begin{bmatrix}
-& x^{(i,1)} & - \\ &\vdots& \\
-&x^{(i,C)} &-
\end{bmatrix} \in \mathbb{R}^{C \times d} \quad \text{ is a } C \times d \text{ matrix}
$$

The labels $y^{(1)},\dots, y^{(n)} \in \{ \pm 1\}$ will indicate two classes (for example, $+1$ means "this sentence has positive sentiment" and $-1$ means "this sentence has negative sentiment").

In the code, we will name
- $C$ as `n_context`
- $d$ as `n_features`
- $n$ as `n_samples`

The variable `X_raw` is a list (of length `n_samples`) of list (of length `n_context`) of lists (of length `n_features`).

In [1]:
n_context = 4
n_features = 3
n_samples = 5

X_raw = [[[-0.707, -0.707, 1.0],
          [0.963, -0.268, 1.0],
          [0.391, 0.92, -1.0],
          [0.899, 0.437, -1.0]],
         [[0.327, -0.945, 1.0],
          [0.3, -0.954, -1.0],
          [-0.485, -0.874, -1.0],
          [-0.694, 0.72, 1.0]],
         [[-0.309, 0.951, -1.0],
          [-0.951, 0.31, 1.0],
          [-0.9, -0.437, 1.0],
          [-0.013, -1.0, -1.0]],
         [[0.829, -0.559, -1.0],
          [-0.856, 0.518, 1.0],
          [-0.2, -0.98, -1.0],
          [-0.842, -0.539, 1.0]],
         [[-0.938, -0.346, 1.0],
          [-0.742, 0.67, -1.0],
          [0.742, 0.67, -1.0],
          [0.322, 0.947, -1.0]]]
y_raw = [-1.0, -1.0, 1.0, 1.0, -1.0]

# The model

Let $q$ be a positive integer (we will take $q=2$ in this problem) smaller than $d$.
You will consider a model of this form:

Let

$$
\theta = [ W^{(1)}, W^{(2)}, w^{(3)}]
$$

where

$$
W^{(1)} \text{ and }  W^{(2)} \in \mathbb{R}^{d \times q}
$$

$$
 w^{(3)} \in \mathbb{R}^{ d} \quad \mbox{is a column vector}
$$

and

$$
f(X^{(i)} ; \theta)
=
\mathrm{softmax} (x^{(i,C)} W^{(1)}  W^{(2) \top} X^{(i) \top} )
X^{(i)}w^{(3)}
$$

Note:
- $x^{(i,C)}$ is the last (bottom) row of $X^{(i)}$ and so $x^{(i,C)\top}$ is a column vector, in particular.
- $ X^{(i)} W^{(2)\top } W^{(1)} x^{(i,C)^\top}$ is a $C$-dimensional vector.
- $f(X^{(i)} ; \theta)$ is a scalar number.

The loss function we will use is 

$$
\log( 1 + \exp(- y^{(i)} f(X^{(i)} ; \theta)))
$$

So the training risk is

$$
J(\theta) = \frac{1}{n} \sum_{i=1}^n \log( 1 + \exp(- y^{(i)} f(X^{(i)} ; \theta)))
$$

This is a special form of the Cross Entropy when the number of classes is equal to $2$, and we when encode ther classes as $\{\pm 1\}$ instead of $\{1,2\}$.

In the code, we will name
- $q$ as `n_reduced`


In [2]:
n_reduced = 2
W1_raw = [[0.74, 0.529], [-0.589, 0.189], [-0.759, -0.933]] # n_features by n_reduced
W2_raw = [[0.504, 0.651], [-0.319, -0.848], [0.606, -2.018]] # n_features by n_reduced
w3_raw = [2.707, 0.628, 0.908] # n_features

## The `ag.Scalar` automatic differentiation library

In [3]:
import math
import matplotlib.pyplot as plt

class ag: # AutoGrad

    def log(input):
        output = ag.Scalar(math.log(input.value), inputs=[input], op="log")

        def _backward():
            input.grad += output.grad / input.value
            return None

        output._backward = _backward
        return output

    def exp(input):

        output = ag.Scalar(math.exp(input.value), inputs=[input], op="exp")

        def _backward():
            input.grad += output.grad * output.value
            return None

        output._backward = _backward
        return output

    def relu(input):
        output = ag.Scalar(max(0, input.value), inputs=[input], op="relu")

        def _backward():
            if input.value > 0:
                input.grad += output.grad
            return None

        output._backward = _backward
        return output

    class Scalar: # Scalars with grads
        def __init__(self,
                     value,
                     op="",
                     _backward= lambda : None,
                     inputs=[],
                     label=""):

            self.value = float(value)
            self.grad = 0.0

            self._backward = _backward
            self.inputs = inputs

            self.op = op
            self.label = label



        def backward(self):
            self.grad = 1.0

            topo_order = self.topological_sort()

            for node in reversed(topo_order):
                node._backward()

        def topological_sort(self):
            topo_order = []
            visited = set()

            def dfs(node):
                if node not in visited:
                    visited.add(node)
                    for input in node.inputs:
                        dfs(input)
                    topo_order.append(node)

            dfs(self)
            return topo_order


        def __add__(self, other):
            if not isinstance(other, ag.Scalar):
                other = ag.Scalar(other, label=f"{other}\nconst")

            output = ag.Scalar(self.value + other.value,
                               inputs=[self, other], op="add")

            def _backward():
                # pass
                self.grad += output.grad
                other.grad += output.grad

            output._backward = _backward
            return output


        def __mul__(self, other):
            if not isinstance(other, ag.Scalar):
                other = ag.Scalar(other, label=f"{other}\nconst")

            output = ag.Scalar(self.value * other.value,
                               inputs=[self, other], op="mul")

            def _backward():

                self.grad += other.value * output.grad
                other.grad += self.value * output.grad

                return None

            output._backward = _backward

            return output
        def __truediv__(self,other):
            return self*(other**(-1))

        def __neg__(self):
            output = ag.Scalar(-self.value, inputs=[self], op="neg")
            def _backward():
                self.grad -= output.grad
                return None
            output._backward = _backward
            return output
        def __sub__(self,other):
            return self + -other

        def __pow__(self, exponent): # exponent is just a python float
            output = ag.Scalar(self.value ** exponent, inputs=[self], op=f"pow({exponent})")

            def _backward():

                self.grad += (exponent * self.value**(exponent-1)) * output.grad
                return None

            output._backward = _backward
            return output

        def __repr__(self) -> str:
            if self.op == "":
                return self.label
            else:
                return self.label + "\n" + self.op


## Problem 3 [10 pts]

Calculate using `ag.Scalar` the following: `dJdw3` `dJdW2` and `dJdW1`.

Hint: 
- It will make your life easier if you build a wrapper library to support matrices and matrix multiplication, using lists of lists of `ag.Scalar`s.
- Convert the (raw) parameter and data matrices to a matrix of `ag.Scalar`s using a function like
  `W1 = toMat(W1_raw)`
- Create a function like `showGrads(W1)` to help you debug.

n_context = C = 4 (words per sentence)

n_features = d = 3 (embedding size)

n_reduced = q = 2 (compressed size)

X(i) = (C*d) = (4x3)

W(1) = (d*q) = (3*2)

W(2) = (d*q) = (3*2)

w(3) = (d) = (3,)

In [4]:
# --- Helper functions for matrices of ag.Scalar ---

def toMat(raw):
    """Convert a list of lists of floats into a list of lists of ag.Scalar."""
    return [[ag.Scalar(val) for val in row] for row in raw]

def toVec(raw):
    """Convert a list of floats into a list of ag.Scalar."""
    return [ag.Scalar(val) for val in raw]

def showGrads(mat):
    """Prints gradients of a matrix or vector of ag.Scalar."""
    if isinstance(mat[0], list):  # matrix
        return [[x.grad for x in row] for row in mat]
    else:  # vector
        return [x.grad for x in mat]

def scalar_sum(values):
    total = ag.Scalar(0.0)
    for v in values:
        total = total + v
    return total

def dot(vec1, vec2):
    """Dot product of two vectors of ag.Scalar."""
    assert len(vec1) == len(vec2)
    return scalar_sum(vec1[i] * vec2[i] for i in range(len(vec1)))


def matvec_mul(mat, vec):
    """Matrix (m×n) times vector (n) → vector (m)."""
    out = []
    for i in range(len(mat)):
        row_sum = ag.Scalar(0.0)
        for j in range(len(vec)):
            row_sum = row_sum + mat[i][j] * vec[j]
        out.append(row_sum)
    return out

def matmul(A, B):
    """Matrix (m×n) times matrix (n×p) → matrix (m×p)."""
    m, n, p = len(A), len(A[0]), len(B[0])
    out = []
    for i in range(m):
        row = []
        for j in range(p):
            s = ag.Scalar(0.0)
            for k in range(n):
                s = s + A[i][k] * B[k][j]
            row.append(s)
        out.append(row)
    return out

def softmax(vec):
    exps = [ag.exp(v) for v in vec]
    sum_exps = scalar_sum(exps)
    return [e / sum_exps for e in exps]


### Convert data and parameters to ag.Scalars

In [5]:
# Convert raw data into Scalars
X = [[toVec(word) for word in sent] for sent in X_raw]
y = toVec(y_raw)

# Convert parameters
W1 = toMat(W1_raw)   # d × q
W2 = toMat(W2_raw)   # d × q
w3 = toVec(w3_raw)   # d


### Forward pass

In [6]:

# code working
losses = []
for i in range(n_samples):
    Xi = X[i]             # C × d
    xi_last = Xi[-1]      # d

    # Step 1: Query = xi_last W1
    W1_T = list(zip(*W1))  # q × d 
    XiW1 = matvec_mul(W1_T, xi_last)  # (1 x d) x (d x q) = (1 x q)
    
    # Step 2: Keys = Xi W2
    XiW2 = matmul(Xi, W2)  # (C x d) * (d * q) = (C × q)

    # Step 3: Scores = q1 ⋅ each row of XiW2
    scores = [dot(XiW1, XiW2[c]) for c in range(n_context)]

    # Step 4: Attention weights
    alpha = softmax(scores) # (4,)

    # Step 5: Prediction
    Xi_w3 = [dot(Xi[c], w3) for c in range(n_context)] 
    f_i = scalar_sum(alpha[c] * Xi_w3[c] for c in range(n_context))  # scalar
     
    print(f"Output forward pass {f_i.value}")

    # Step 6: Logistic loss
    y_i = y[i]
    loss_i = ag.log(ag.Scalar(1.0) + ag.exp(-y_i * f_i))
    
    losses.append(loss_i)

# Average loss
J = scalar_sum(losses) * (1.0 / n_samples)



Output forward pass 1.652897696741296
Output forward pass -0.5030430458474209
Output forward pass -1.5462990368860332
Output forward pass -1.1800069124499617
Output forward pass 0.745286662125376


### backprop & gradient extraction

In [7]:
# Backward
J.backward()

# Extract gradients
dJdw3 = showGrads(w3)
dJdW2 = showGrads(W2)
dJdW1 = showGrads(W1)

print("dJdw3 =", dJdw3)
print("dJdW2 =", dJdW2)
print("dJdW1 =", dJdW1)


dJdw3 = [0.2900661907172973, 0.31213455010706614, -0.22591685346105916]
dJdW2 = [[0.05486582485602198, 0.1340124413293936], [-0.01348159334764005, -0.009055390598639454], [0.014211207749059897, 0.014788313958514373]]
dJdW1 = [[0.02673784282047291, 0.011368294676590236], [0.04348804004907827, 0.05114747020150545], [-0.06112443715812872, -0.05294485874898042]]


## Check your answers

In [8]:
# output of forward
# [ 1.6528977 , -0.50304305, -1.54629904, -1.18000691,  0.74528666]

# dJdw3
# [ 0.29006619,  0.31213455, -0.22591685]

# dJdW2
# [[ 0.05486582,  0.13401244],
#  [-0.01348159, -0.00905539],
#  [ 0.01421121,  0.01478831]]


# dJdW1
# [[ 0.02673784,  0.01136829],
#  [ 0.04348804,  0.05114747],
#  [-0.06112444, -0.05294486]]